lean version of read in file for the NMX work flow (from simulatet event data to binned event data)


first add neded libaries

In [2]:
from datetime import datetime
startTime = datetime.now()
import tracemalloc
    #from scippnexus import data
#import scippnexus as snx
import numpy as np
    #qimport mantid_args
#from scippnexus import NXdetector
from scippneutron.conversion import graph

import scipp as sc
import scippneutron as scn
%matplotlib widget
import plopp as pp
pp.patch_scipp()
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

    #import multiprocessing as mp

import h5py
h5py.enable_ipython_completer()


In [3]:
#%reset out
sys.version


'3.9.15 | packaged by conda-forge | (main, Nov 22 2022, 08:52:10) \n[Clang 14.0.6 ]'

In [3]:
#print(Out)

In [4]:
# starting the momory monitoring
tracemalloc.start()

pretreatment to make it faster
h5repack -l CHUNK=1024x6 2e11.h5 2e11-rechunk.h5 
or 
h5repack -l CHUNK=NONE 2e11.h5 2e11-nochunk.h5

h5repack -l CHUNK=1024x6 mccode.h5 mccode-nochunk.h5

In [5]:
fname = 9
if fname == 1:
   filename = '/Users/justinbergmann/work_flow/test_data/2e11.h5'

elif fname == 2:
    filename = '/Users/justinbergmann/work_flow/test_data/update_det/5e9/x_0_NMX/mccode.h5'

elif fname == 3:
    filename = '/Users/justinbergmann/work_flow/test_data/small_det/xe_1_NMX/mccode.h5'

#elif fname == 4:
 #   filename = '/Users/justinbergmann/work_flow/test_data/small_det/xe_2_NMX/mccode.h5'

elif fname == 5:
    filename = '/Users/justinbergmann/work_flow/test_data/xe1_5_NMX/small_det.h5'
elif fname == 6:
    filename = '/Users/justinbergmann/work_flow/test_data/xe2_10_NMX/mccode.h5'
elif fname == 7:
    filename = '/Users/justinbergmann/work_flow/test_data/xe5_10_NMX/medium_5e10.h5'
elif fname == 8:
    filename = '/Users/justinbergmann/work_flow/test_data/xe1_12_NMX/xe1_12_no_chunk.h5'
elif fname == 9:
    filename = '/Users/justinbergmann/work_flow/test_data/1d.h5'
else:
    filename = '/Users/justinbergmann/work_flow/test_data/e12_no_chunk.h5'

#f=snx.File(filename)    

read in event data and give number of events

In [6]:
%%time
f = h5py.File(filename)
a = f['entry1/data']['bank01_events_dat_list_p_x_y_n_id_t']['events'][...]
a[0]

CPU times: user 320 ms, sys: 592 ms, total: 911 ms
Wall time: 1.11 s


array([6.21507181e-05, 2.14133618e-01, 7.89053704e-02, 0.00000000e+00,
       2.52696000e+05, 1.21962647e-01])

In [7]:
#sc.show_graph(graph.tof.kinematic("tof"), simplified=True)

In [7]:
print(f.filename)
print(f.file)
print(f.driver)
print(f.mode)


/Users/justinbergmann/work_flow/test_data/1d.h5
<HDF5 file "1d.h5" (mode r)>
sec2
r


In [8]:
print(a.shape)
d = np.matrix.transpose(a)
print(d.shape)

(62426271, 6)
(6, 62426271)


In [9]:
x_list = sc.array(dims=['x'], unit='m', values=d[1])
y_list = sc.array(dims=['x'], unit='m', values=d[2])
t_list = sc.array(dims=['x'], unit='ms', values=d[5])
id_list = sc.array(dims=['x'], unit='none', values=d[4])
print(x_list.shape, y_list.shape, t_list.shape,id_list.shape)

(62426271,) (62426271,) (62426271,) (62426271,)


get evetns to seperated lists (x,y,t)

In [10]:
#x_list.unit = 'm'
#y_list.unit = 'm'
#t_list.unit = 'ms'
weights = sc.ones_like(x_list)
weights.unit = 'counts'
da = sc.DataArray(data=weights, coords={'x': x_list, 'y': y_list, 't': t_list, 'id': id_list})
print(da)

<scipp.DataArray>
Dimensions: Sizes[x:62426271, ]
Coordinates:
  id                        float64          [1e-09]  (x)  [252696, 791703, ..., 443080, 1.23049e+06]
  t                         float64             [ms]  (x)  [0.121963, 0.121914, ..., 0.11844, 0.11842]
  x                         float64              [m]  (x)  [0.214134, 0.265178, ..., 0.0799815, 0.162174]
  y                         float64              [m]  (x)  [0.0789054, 0.247318, ..., 0.138776, 0.38467]
Data:
                            float64         [counts]  (x)  [1, 1, ..., 1, 1]




In [11]:
id_list.max

<bound method max of <scipp.Variable> (x: 62426271)    float64          [1e-09]  [252696, 791703, ..., 443080, 1.23049e+06]>

dd 2D binning (x and y)

In [13]:
nbin= 1280
binned = da.bin(y=nbin, x=nbin)
#binned = da.bin(y=50, x=50)
#binned = da.bin(y=225, x=225)
#sc.transpose(binned)

binned

<scipp.DataArray>
Dimensions: Sizes[x:1280, y:1280, ]
Coordinates:
  x                         float64              [m]  (x [bin-edge])  [3.90616e-09, 0.000400004, ..., 0.5116, 0.512]
  y                         float64              [m]  (y [bin-edge])  [1.72307e-08, 0.000400017, ..., 0.5116, 0.512]
Data:
                          DataArrayView        <no unit>  (x, y)  [<scipp.DataArray>
Dimensions: Sizes[x:7, ]
Coordinates:
  id                        float64          [1e-09]  (x)  [1, 1, ..., 1, 1]
  t                         float64             [ms]  (x)  [0.130026, 0.111474, ..., 0.138933, 0.106004]
  x                         float64              [m]  (x)  [0.000111383, 0.000356121, ..., 0.000357237, 0.000349801]
  y                         float64              [m]  (x)  [3.49796e-05, 5.81792e-05, ..., 0.00019046, 0.00013648]
Data:
                            float64         [counts]  (x)  [1, 1, ..., 1, 1]

, <scipp.DataArray>
Dimensions: Sizes[x:11, ]
Coordinates:
  id                        float64          [1e-09]  (x)  [1281, 1281, ..., 1281, 1281]
  t                         float64             [ms]  (x)  [0.119567, 0.114171, ..., 0.118675, 0.127567]
  x                         float64              [m]  (x)  [0.000328752, 0.000127446, ..., 0.000201739, 0.000295327]
  y                         float64              [m]  (x)  [0.000781176, 0.000695529, ..., 0.000784756, 0.000510317]
Data:
                            float64         [counts]  (x)  [1, 1, ..., 1, 1]

, ..., <scipp.DataArray>
Dimensions: Sizes[x:10, ]
Coordinates:
  id                        float64          [1e-09]  (x)  [1.63712e+06, 1.63712e+06, ..., 1.63712e+06, 1.63712e+06]
  t                         float64             [ms]  (x)  [0.109787, 0.126664, ..., 0.10762, 0.0995295]
  x                         float64              [m]  (x)  [0.511799, 0.5117, ..., 0.511968, 0.511984]
  y                         float64              [m]  (x)  [0.511449, 0.511229, ..., 0.511524, 0.511409]
Data:
                            float64         [counts]  (x)  [1, 1, ..., 1, 1]

, <scipp.DataArray>
Dimensions: Sizes[x:5, ]
Coordinates:
  id                        float64          [1e-09]  (x)  [1.6384e+06, 1.6384e+06, ..., 1.6384e+06, 1.6384e+06]
  t                         float64             [ms]  (x)  [0.10841, 0.10724, ..., 0.132072, 0.11102]
  x                         float64              [m]  (x)  [0.511938, 0.511608, ..., 0.511814, 0.511847]
  y                         float64              [m]  (x)  [0.512, 0.511764, ..., 0.511775, 0.511927]
Data:
                            float64         [counts]  (x)  [1, 1, ..., 1, 1]

]

plot binned data, histogrammed over all times

In [14]:
fig = binned.hist().transpose().plot(aspect='equal')
fig

InteractiveFig2d(children=(HBar(), HBox(children=(VBar(children=(Toolbar(children=(ButtonTool(icon='home', lay…

3D binning

In [15]:
tbin=250
hist3=binned.hist(t=tbin)

histrogramm 3D data


plot t slice

In [16]:
nb=20
times=hist3.coords['t']
print(len(times))
print(len(times),times[nb],times[nb+1])
hist3['t',nb].transpose().plot(aspect='equal')

251
251 <scipp.Variable> ()    float64             [ms]  [0.0975479] <scipp.Variable> ()    float64             [ms]  [0.0977835]


InteractiveFig2d(children=(HBar(), HBox(children=(VBar(children=(Toolbar(children=(ButtonTool(icon='home', lay…

nn with pixel id

In [17]:
hist3

<scipp.DataArray>
Dimensions: Sizes[x:1280, y:1280, t:250, ]
Coordinates:
  t                         float64             [ms]  (t [bin-edge])  [0.092834, 0.0930697, ..., 0.151522, 0.151757]
  x                         float64              [m]  (x [bin-edge])  [3.90616e-09, 0.000400004, ..., 0.5116, 0.512]
  y                         float64              [m]  (y [bin-edge])  [1.72307e-08, 0.000400017, ..., 0.5116, 0.512]
Data:
                            float64         [counts]  (x, y, t)  [0, 0, ..., 0, 0]

In [18]:
# del hist3.coords['tc']
# del hist3.coords['xc']
# del hist3.coords['yc']

In [18]:
hist4 = hist3.copy()
for name in list(hist4.coords.keys()):
    hist4.coords[f'{name}c'] = sc.midpoints(hist4.coords[name])
hist4

<scipp.DataArray>
Dimensions: Sizes[x:1280, y:1280, t:250, ]
Coordinates:
  t                         float64             [ms]  (t [bin-edge])  [0.092834, 0.0930697, ..., 0.151522, 0.151757]
  tc                        float64             [ms]  (t)  [0.0929518, 0.0931875, ..., 0.151404, 0.151639]
  x                         float64              [m]  (x [bin-edge])  [3.90616e-09, 0.000400004, ..., 0.5116, 0.512]
  xc                        float64              [m]  (x)  [0.000200004, 0.000600004, ..., 0.5114, 0.5118]
  y                         float64              [m]  (y [bin-edge])  [1.72307e-08, 0.000400017, ..., 0.5116, 0.512]
  yc                        float64              [m]  (y)  [0.000200017, 0.000600017, ..., 0.5114, 0.5118]
Data:
                            float64         [counts]  (x, y, t)  [0, 0, ..., 0, 0]

In [20]:
300*300*200

18000000

In [29]:
hist5 = sc.flatten(hist4, to='row')
select = hist5.data > sc.scalar(5, unit='counts') # cut of background
filtered = hist5[select]
filtered.coords['tc'] *= 10
filtered

<scipp.DataArray>
Dimensions: Sizes[row:2279, ]
Coordinates:
  tc                        float64             [ms]  (row)  [1.38912, 1.25477, ..., 1.16521, 1.19349]
  xc                        float64              [m]  (row)  [0.0422, 0.0778, ..., 0.4258, 0.4294]
  yc                        float64              [m]  (row)  [0.2498, 0.1778, ..., 0.2282, 0.249]
Data:
                            float64         [counts]  (row)  [6, 6, ..., 6, 6]

In [54]:
hist5

<scipp.DataArray>
Dimensions: Sizes[row:409600000, ]
Coordinates:
  tc                        float64             [ms]  (row)  [0.0929518, 0.0931875, ..., 0.151404, 0.151639]
  xc                        float64              [m]  (row)  [0.000200004, 0.000200004, ..., 0.5118, 0.5118]
  yc                        float64              [m]  (row)  [0.000200017, 0.000200017, ..., 0.5118, 0.5118]
Data:
                            float64         [counts]  (row)  [0, 0, ..., 0, 0]

In [30]:
pp.scatter3d(filtered, x='xc', y='yc', z='tc', pixel_size=0.001)#,figsize=(550, 550))

Box(children=(InteractiveFig3d(children=(HBar(), HBox(children=(VBar(children=(Toolbar(children=(ButtonTool(ic…

In [23]:

#pp.scatter3d(hist3, x='x', y='y', z='t', figsize=(550, 400), pixel_size=0.2)


In [24]:
%matplotlib widget
print(hist3.max().value)
pp.slicer(hist3, keep=['x', 'y'],vmax=hist3.max().value)


65.0


Box(children=(InteractiveFig2d(children=(HBar(), HBox(children=(VBar(children=(Toolbar(children=(ButtonTool(ic…

In [64]:
nbin= 1280**2
tbin = 12
id_binned = da.bin(id=nbin)

id_hist=id_binned.hist(t=tbin)


id_hist.sum

<bound method sum of <scipp.DataArray>
Dimensions: Sizes[id:1638400, t:12, ]
Coordinates:
  id                        float64          [1e-09]  (id [bin-edge])  [1, 2, ..., 1.6384e+06, 1.6384e+06]
  t                         float64             [ms]  (t [bin-edge])  [0.092834, 0.0977443, ..., 0.146847, 0.151757]
Data:
                            float64         [counts]  (id, t)  [0, 0, ..., 0, 0]

>

In [26]:
#f_out = h5py.File('/Users/justinbergmann/work_flow/test_out/test.h5','w')

In [65]:
print(id_hist['t',0][0].shape)
print(id_hist.shape)
data1 = np.array(id_hist['t',0].values)
data2 = np.array(id_hist.values)
data_t = np.array(id_hist['id',0])
print(id_hist['t',0])
print(data1.shape)
id_hist

()
(1638400, 12)
<scipp.DataArray>
Dimensions: Sizes[id:1638400, ]
Coordinates:
  id                        float64          [1e-09]  (id [bin-edge])  [1, 2, ..., 1.6384e+06, 1.6384e+06]
Data:
                            float64         [counts]  (id)  [0, 0, ..., 1, 0]
Attributes:
  t                         float64             [ms]  (t [bin-edge])  [0.092834, 0.0977443]


(1638400,)


<scipp.DataArray>
Dimensions: Sizes[id:1638400, t:12, ]
Coordinates:
  id                        float64          [1e-09]  (id [bin-edge])  [1, 2, ..., 1.6384e+06, 1.6384e+06]
  t                         float64             [ms]  (t [bin-edge])  [0.092834, 0.0977443, ..., 0.146847, 0.151757]
Data:
                            float64         [counts]  (id, t)  [0, 0, ..., 0, 0]

In [76]:
t_list = sc.midpoints(id_hist.coords['t'])
t_list_n = np.array(t_list.values)
print(len(t_list_n))
#print(t_list_n)
tl = np.array(sc.midpoints(id_hist.coords['t']).values) 
print(tl)

12
[0.09528913 0.1001994  0.10510968 0.11001995 0.11493023 0.1198405
 0.12475078 0.12966105 0.13457133 0.1394816  0.14439188 0.14930215]


In [104]:
tl_bin = np.array(id_hist.coords['t'].values)
id_bin = np.array(id_hist.coords['id'].values)
id_r = np.rint(id_bin)
#id_r= np.int(id_r)
print(len(tl_bin))
print(tl_bin)
print(len(id_bin)-(1280**2))
print(id_bin)
print(type(id_r[0]))

13
[0.09283399 0.09774426 0.10265454 0.10756481 0.11247509 0.11738536
 0.12229564 0.12720591 0.13211619 0.13702646 0.14193674 0.14684701
 0.15175729]
1
[1.00000000e+00 1.99999939e+00 2.99999878e+00 ... 1.63839800e+06
 1.63839900e+06 1.63840000e+06]
<class 'numpy.float64'>


In [43]:
sc.show(id_hist)

In [50]:
id_hist.get('t')

AttributeError: 'scipp._scipp.core.DataArray' object has no attribute 'get'

In [28]:
print(data1.shape)
print(data1[:20])

(1638400,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [46]:
print(id_hist['id',20])

<scipp.DataArray>
Dimensions: Sizes[t:12, ]
Coordinates:
  t                         float64             [ms]  (t [bin-edge])  [0.092834, 0.0977443, ..., 0.146847, 0.151757]
Data:
                            float64         [counts]  (t)  [0, 0, ..., 1, 0]
Attributes:
  id                        float64          [1e-09]  (id [bin-edge])  [21, 22]




In [ ]:
print(id_hist.shape)
print(id_hist['t',1].shape)
print(len(data1))
print(data1[0])

In [ ]:
print(data1.dtype)

In [ ]:
#grp = f_out.create_group("raw_data_1")

#grp.__setitem__('beamline','NMX')
#grp.__setitem__('definition','TOFRAW')
#det1 = grp.create_group("detector_1")
#dset = det1.create_dataset("hist_data",data=data1)

#beamline = grp.create_dataset("beamline", ("NMX"))
#print(beamline.name)
#beamline = "NMX"
#"NMX" = grp['beamline'] 

In [91]:
with h5py.File('/Users/justinbergmann/work_flow/test_out/test.h5', 'w') as fo:
## create output nexus file
    fo.attrs['default'] = 'entry'
    nxentry = fo.create_group('entry')
    nxentry.attrs["NX_class"] = 'NXentry'
    nxentry.attrs['default'] = 'data'
    nxentry.__setitem__('beamline','NMX')
    nxentry.__setitem__('definition','TOFRAW')


    nxdata = nxentry.create_group('data')
    nxdata.attrs["NX_class"] = 'NXdata'
    nxdata.attrs['signal'] = 'counts'
    nxdata.attrs['axes'] = 'two_theta'
    nxdata.attrs['two_theta_indices'] = [0,]
 
    tth = nxdata.create_dataset('data1', data=data1)
    tth.attrs['units'] = 'counts'
    tth.attrs['long_name'] = 'data1 (counts)'

    id_bin = nxdata.create_dataset('id_bin', data=id_bin)
    id_bin.attrs['units'] = ''
    id_bin.attrs['long_name'] = 'pixel ID'
    
    t_bin = nxdata.create_dataset('t_bin', data=tl_bin)
    t_bin.attrs['units'] = 'ms'
    t_bin.attrs['long_name'] = 't_bin TOF (ms)'

    counts = nxdata.create_dataset('counts', data=data2)
    counts.attrs['units'] = 'counts'
    counts.attrs['long_name'] = 'counts (bined counts)'

    fo.close()


In [81]:
#print(grp['beamline'].items())
fo.close()

In [ ]:

#f_out.close()

In [ ]:
f.close()  
del id_list
del x_list
del y_list
del t_list
del id_hist
del hist3

In [1]:
current, peak = tracemalloc.get_traced_memory()
print('2 current memory [MB]: {}, peak memory [MB]: {} '.format(round((current/(1024*1024)), 2), round((peak /(1024*1024) ), 2) ))
# stopping the library
tracemalloc.stop()
print("neded time (h:mm:ss): ", datetime.now() - startTime)

NameError: name 'tracemalloc' is not defined